<a href="https://colab.research.google.com/github/cfarhutchins/technical_direction_service/blob/master/BOW_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install GetOldTweets3
!pip install vaderSentiment

     |████████████████████████████████| 133kB 5.1MB/s 


In [3]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils
import GetOldTweets3 as got
from sklearn.feature_extraction.text import CountVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import nltk 
import bs4 as BeautifulSoup
nltk.download('stopwords')
import html5lib


Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('reparations')\
                                           .setSince('2019-07-04')\
                                           .setUntil('2019-07-11')\
                                           .setMaxTweets(30_000)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)

repar_tweets = []

for i in tweet:
  repar_tweets.append(i.text)

In [0]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('reparations')\
                                           .setSince('2019-06-13')\
                                           .setUntil('2019-06-20')\
                                           .setMaxTweets(30_000)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)



for i in tweet:
  repar_tweets.append(i.text)

In [6]:
len(repar_tweets)

60000

In [0]:
analyzer = SentimentIntensityAnalyzer()

In [0]:
df_repar = pd.DataFrame(repar_tweets)
df_repar.rename(columns={0:'tweet'}, inplace= True)
df_repar['feels'] = df_repar['tweet'].apply(lambda i: analyzer.polarity_scores(i))
compound = []

for dct in df_repar['feels']:
  compound.append(dct['compound'])

negative = []
  
for dct in df_repar['feels']:
  negative.append(dct['neg'])
  
neutral = [] 
  
for dct in df_repar['feels']:
  neutral.append(dct['neu'])
  
positive = []

for dct in df_repar['feels']:
  positive.append(dct['pos'])
  
df_repar['compound'] = compound
df_repar['negative'] = negative
df_repar['neutral'] = neutral
df_repar['positive'] = positive

df_repar.drop(columns = 'feels', inplace = True)

In [9]:
df_repar.head()

,tweet,compound,negative,neutral,positive
0,I don’t think they deserve reparations they st...,-0.4939,0.186,0.814,0.000
1,"If you listen to Rush, you’d already know that...",0.1027,0.000,0.949,0.051
2,Uber reparations: my new passion project,0.4588,0.000,0.625,0.375
3,What I’m about to say may and may not offend y...,0.7626,0.000,0.856,0.144
4,@JacksonLeeTX18 #ados don’t deserve this we ar...,-0.8257,0.181,0.749,0.070


In [0]:
from nltk.corpus import stopwords
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import column_or_1d

In [0]:
word = re.compile('[^\s]+')


In [12]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    #text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df_repar['clean_text'] = df_repar['tweet'].apply(clean_text)
#print_plot(10)
df_repar.head()

,tweet,compound,negative,neutral,positive,clean_text
0,I don’t think they deserve reparations they st...,-0.4939,0.186,0.814,0.000,dont think deserve reparations steal enough ma...
1,"If you listen to Rush, you’d already know that...",0.1027,0.000,0.949,0.051,listen rush youd already know guess shell payi...
2,Uber reparations: my new passion project,0.4588,0.000,0.625,0.375,uber reparations new passion project
3,What I’m about to say may and may not offend y...,0.7626,0.000,0.856,0.144,im say may may offend #isreal done people amer...
4,@JacksonLeeTX18 #ados don’t deserve this we ar...,-0.8257,0.181,0.749,0.070,jacksonleetx18 #ados dont deserve still slaves...


In [0]:
my_tags = ['reparations',
           'slavery',
           'descendants', 
           'stupid',
           'civil war',
           'ancestors',
           'dead',
           'hell',
           'racism',
           'abuse',
           'shit',
           'bitch',
           'bullshit',
           'vote',
           'pay',
           'pain',
           'suffering',
           'illegal aliens',
           'dead',
           'scam',
           'chattel', 
           'forced',
           'stealing',
           'end',
           'stealth',
           'good',
           'luck', 
           'happy',
           'win', 
           'grandfathers',
           'grandmothers',
           'honestly',
           'need',
           'better',
           'freedom',
           'wealth', 
           'redistribution',
           'support',
           'public',
           'forum',
           'argument',
           'peace',
           'love',
           'justice',
           'claim'
]


In [14]:
pingbit = pd.Series(my_tags)
pingbit.head()

0    reparations
1        slavery
2    descendants
3         stupid
4      civil war
dtype: object

In [0]:
cv = CountVectorizer(ngram_range=(2,2),binary=True, stop_words='english')


In [0]:
x = df_repar['tweet']
y = df_repar['compound']



In [0]:
from sklearn.model_selection import train_test_split, cross_val_score

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= .33)

X_train = cv.fit_transform(x_train)

X_test = cv.transform(x_test)

In [0]:
from sklearn.linear_model import Ridge

rdg = Ridge()

In [20]:
rdg.fit(X_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [21]:
rdg.predict(X_test)

array([-0.01782127, -0.38748867,  0.36258335, ...,  0.00858623,
        1.11527404, -0.16980367])

In [0]:
rdg.score(X_test, y_test)
pingwords = cv.get_feature_names()

In [0]:
df_pingwords = pd.DataFrame(rdg.coef_, index = pingwords).sort_values(by=0)


In [0]:
ping_scores = []

for data in df_pingwords[0]:
  ping_scores.append(data)

df_ping_scores = pd.DataFrame(ping_scores)

In [25]:
df_ping_scores.head()

,0
0,-0.478084
1,-0.477979
2,-0.463342
3,-0.459906
4,-0.458670


In [0]:
pingwords_lst = []

for i in pingwords:
  pingwords_lst.append(i)

In [0]:
df_pingwords = df_pingwords.index
  

In [0]:
pingwords_lst = df_pingwords.to_list()

In [0]:
df_pingwords = pd.DataFrame(pingwords_lst)

In [30]:
df_pingwords

,0
0,reparations bullshit
1,pain suffering
2,reparations stupid
3,reparations slavery
4,slavery reparations
5,reparations hell
6,descendants slavery
7,justice happy
8,started slavery
9,civil war


In [0]:
df_pingwords.rename(columns={0:'ping pair'}, inplace= True)

In [32]:
df_pingwords.head()

,ping pair
0,reparations bullshit
1,pain suffering
2,reparations stupid
3,reparations slavery
4,slavery reparations


In [0]:
df_attention = df_pingwords.join(df_ping_scores)
df_attention.rename(columns = {0:'score'}, inplace=True)
#df_pingwords.rename(columns={0:'attention'})

In [34]:
df_attention.head()

,ping pair,score
0,reparations bullshit,-0.478084
1,pain suffering,-0.477979
2,reparations stupid,-0.463342
3,reparations slavery,-0.459906
4,slavery reparations,-0.458670


In [0]:
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import column_or_1d

In [0]:
class TolerantLabelEncoder(LabelEncoder):
    def __init__(self, ignore_unknown=False,
                       unknown_original_value='unknown', 
                       unknown_encoded_value=-1):
        self.ignore_unknown = ignore_unknown
        self.unknown_original_value = unknown_original_value
        self.unknown_encoded_value = unknown_encoded_value

    def transform(self, y):
        check_is_fitted(self, 'classes_')
        y = column_or_1d(y, warn=True)

        indices = np.isin(y, self.classes_)
        if not self.ignore_unknown and not np.all(indices):
            raise ValueError("y contains new labels: %s" 
                                         % str(np.setdiff1d(y, self.classes_)))

        y_transformed = np.searchsorted(self.classes_, y)
        y_transformed[~indices]=self.unknown_encoded_value
        return y_transformed

    def inverse_transform(self, y):
        check_is_fitted(self, 'classes_')

        labels = np.arange(len(self.classes_))
        indices = np.isin(y, labels)
        if not self.ignore_unknown and not np.all(indices):
            raise ValueError("y contains new labels: %s" 
                                         % str(np.setdiff1d(y, self.classes_)))

        y_transformed = np.asarray(self.classes_[y], dtype=object)
        y_transformed[~indices]=self.unknown_original_value
        return y_transformed

In [37]:
train_size = int(len(df_repar) * .7)
train_posts = df_repar['clean_text'][:train_size]
train_tags = df_repar['compound'][:train_size]

test_posts = df_repar['clean_text'][train_size:]
test_tags = df_repar['compound'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)


encoder = TolerantLabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.fit_transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 2

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Adagrad',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

W0712 15:19:15.323816 140703541794688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0712 15:19:15.362080 140703541794688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0712 15:19:15.371611 140703541794688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0712 15:19:15.401221 140703541794688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0712 15:19:15.414145 

Train on 37800 samples, validate on 4200 samples
Epoch 1/2
37800/37800 [==============================] - 81s 2ms/step - loss: 5.6873 - acc: 0.2560 - val_loss: 5.3404 - val_acc: 0.2802
Epoch 2/2
37800/37800 [==============================] - 78s 2ms/step - loss: 5.1644 - acc: 0.2725 - val_loss: 5.2974 - val_acc: 0.2857


In [38]:
len(repar_tweets)

60000

In [0]:
history.on_epoch_end(epoch= 'Epoch 12/12')

In [40]:

history.model.predict(x_test)

#model.predict(x_test)

array([[3.91564972e-05, 3.58275393e-05, 2.35838488e-05, ...,
        2.69744978e-05, 4.92191466e-05, 6.71449307e-05],
       [3.59915102e-05, 4.68307553e-05, 2.66431452e-05, ...,
        3.83415318e-05, 7.41149343e-05, 5.05082171e-05],
       [3.63293802e-05, 6.35521283e-05, 2.45162391e-05, ...,
        3.16697369e-05, 8.08040131e-05, 4.19429853e-05],
       ...,
       [4.00537210e-05, 5.84480767e-05, 2.74587728e-05, ...,
        3.56572818e-05, 4.84936681e-05, 3.66787244e-05],
       [8.68078059e-05, 1.07069296e-04, 5.92396282e-05, ...,
        7.90431732e-05, 6.66795167e-05, 9.26369394e-05],
       [1.88120339e-06, 4.81364441e-06, 1.23867017e-06, ...,
        1.68481779e-06, 4.27724262e-06, 1.79644564e-06]], dtype=float32)

In [41]:
score = history.model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Test accuracy:', score[0])

18000/18000 [==============================] - 7s 393us/step
Test accuracy: 11.267915066189236


In [42]:
df_repar.head()

,tweet,compound,negative,neutral,positive,clean_text
0,I don’t think they deserve reparations they st...,-0.4939,0.186,0.814,0.000,dont think deserve reparations steal enough ma...
1,"If you listen to Rush, you’d already know that...",0.1027,0.000,0.949,0.051,listen rush youd already know guess shell payi...
2,Uber reparations: my new passion project,0.4588,0.000,0.625,0.375,uber reparations new passion project
3,What I’m about to say may and may not offend y...,0.7626,0.000,0.856,0.144,im say may may offend #isreal done people amer...
4,@JacksonLeeTX18 #ados don’t deserve this we ar...,-0.8257,0.181,0.749,0.070,jacksonleetx18 #ados dont deserve still slaves...


In [53]:
df_attention[294059:294079]

,ping pair,score
294059,history means,0.339130
294060,lol reparations,0.346998
294061,great great,0.351103
294062,god bless,0.358599
294063,love reparations,0.363810
294064,joke https,0.371377
294065,reparations lmao,0.374410
294066,best reparations,0.382070
294067,great reparations,0.386090
294068,means slavery,0.387728


In [46]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [0]:
df_attention.to_csv('reparations_pingwords.csv')
!cp reparations_pingwords.csv drive/My\ Drive/
